In [3]:
#Dependencies
%matplotlib inline
import os
import csv
import pandas as pd
import psycopg2 as ps2
import matplotlib.pyplot as plt
import datetime as date
from sqlalchemy import create_engine, inspect, func
from sqlalchemy.orm import session
from config import password

In [16]:
#Read CSV files
file = "Resources.LL/3_cases_and_deaths_by_state_timeseries.csv"
case_state = pd.read_csv(file, parse_dates = ["date"], usecols= ["state", "date", "total_population", "cumulative_cases", "cumulative_deaths"])
new_case_state= case_state.dropna(how= "any")
new_case_state.head()

,state,date,total_population,cumulative_cases,cumulative_deaths
0,Alabama,2020-01-22,4887871.0,0,0
1,Alabama,2020-01-23,4887871.0,0,0
2,Alabama,2020-01-24,4887871.0,0,0
3,Alabama,2020-01-25,4887871.0,0,0
4,Alabama,2020-01-26,4887871.0,0,0


In [11]:
case_state.dtypes

state                        object
date                 datetime64[ns]
total_population            float64
cumulative_cases              int64
cumulative_deaths             int64
dtype: object

In [12]:
#Create 
rds_connection_string = "postgres:Cucumber#88@localhost:5432/Covid_John_Hopkins"
engine = create_engine(f"postgresql://{rds_connection_string}")

In [14]:
engine.table_names()

C:\Users\twozi\AppData\Local\Temp/ipykernel_10900/4162273999.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['case_state']

In [17]:
new_case_state.to_sql(name = "case_state", con= engine, if_exists = "append", index = False)

In [19]:
pd.read_sql_query("select * from case_state", con = engine).head()

,state,date,total_population,cumulative_cases,cumulative_deaths
0,Alabama,2020-01-22,4887871.0,0,0
1,Alabama,2020-01-23,4887871.0,0,0
2,Alabama,2020-01-24,4887871.0,0,0
3,Alabama,2020-01-25,4887871.0,0,0
4,Alabama,2020-01-26,4887871.0,0,0


In [20]:
#Read 2nd source of data from Kam
file2 = "Resources.kd/covid-19-cases.csv"

In [23]:
covid19_cases = pd.read_csv(file2, parse_dates = ["Date"], usecols= ["Cases", "Date", "Country_Region", "Province_State"])
covid19_cases.head()

,Cases,Date,Country_Region,Province_State
0,319,2020-04-11,Mauritius,NaN
1,0,2020-02-27,Bahamas,NaN
2,16,2020-02-16,Germany,NaN
3,0,2020-01-31,Canada,Alberta
4,0,2020-03-04,Australia,Queensland


In [24]:
new_covid19_cases= covid19_cases.dropna(how= "any")
new_covid19_cases.head()

,Cases,Date,Country_Region,Province_State
3,0,2020-01-31,Canada,Alberta
4,0,2020-03-04,Australia,Queensland
10,6,2020-02-24,China,Hebei
16,0,2020-02-23,Netherlands,Sint Maarten
17,19665,2020-02-05,China,Hubei


In [31]:
new_covid19_cases.to_sql(name = "covid19_cases", con= engine, if_exists = "append", index = False)

In [32]:
pd.read_sql_query("select * from covid19_cases", con = engine).head()

,Cases,Date,Country_Region,Province_State
0,0,2020-01-31,Canada,Alberta
1,0,2020-03-04,Australia,Queensland
2,6,2020-02-24,China,Hebei
3,0,2020-02-23,Netherlands,Sint Maarten
4,19665,2020-02-05,China,Hubei
